## Introduction



We&rsquo;re thinking about the problem of finding the cheapest possible
nutritious diet.  Last time we argued that this problem could be
expressed as a *linear program*
$$
    \min_x p'x
$$

such that
$$\begin{bmatrix}
      A\\
      -A
   \end{bmatrix}x \geq \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix},$$

where $p$ is a vector of prices, $A$ is a matrix that maps
vectors of quantities of food into vectors of nutrients, and where
$b_{min}$ and $b_{max}$ are respectively dietary minimums
and maximums of different nutrients.  As above, we will sometimes stack these
objects, obtaining
$$
      \tilde{A} = \begin{bmatrix}
                        A_{min}\\
                        -A_{max}
                      \end{bmatrix}
  $$
and
$$
      \tilde{b} = \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix}
  $$

Our job in this notebook: Specify the objects required by the linear
program $(p,\tilde{A},\tilde{b})$, then have the computer solve the problem for us.



## Setup



We need some particular versions of the following modules;



In [1]:
!pip install -r requirements.txt

## USDA Food Central DataBase



The USDA maintains a database of nutritional information, where
  different kinds of food are identified by an FDC number.  They do
  not provide any data on prices.  

To look up nutritional information, use api provided by the USDA at
[https://fdc.nal.usda.gov/](https://fdc.nal.usda.gov/).   You should sign up for a
free api key (see directions on page), then add that key here in
place of &ldquo;DEMO<sub>KEY</sub>&rdquo;.



In [1]:
apikey = "DEMO_KEY"  # Replace with a real key!  "DEMO_KEY" will be slow...

### Looking up foods



I&rsquo;ve written a little module `fooddatacentral` with the methods

-   `search`
-   `nutrients`
-   `units`



### FDC Search



Here&rsquo;s a little code to help look up FDC codes for foods of
different descriptions.



In [1]:
import fooddatacentral as fdc

fdc.search(apikey,"crunchy peanut butter")

### FDC Nutrients



Once we know the `fdc_id` of a particular food we can look up a
variety of information on it.  We start with nutrients



In [1]:
id =     # Put an FDC ID HERE!
fdc.nutrients(apikey,fdc_id=id)

## Prices



Now, let&rsquo;s begin thinking about constructing the objects we need for
the linear program.  Start with specifying $p$, the vector of prices.  

Also note that some kinds of foods need to have unit weights (in
grams) supplied under &ldquo;Units&rdquo;; e.g., extra large eggs are taken to
each weigh 56g.  These conversions can also often be found on the USDA
FDC website.  

Food is purchased in particular units (gallons, pounds, grams).  And
in some cases the natural units are things like donuts or eggs, in
which case we may need to define our  own units (see the example of
&ldquo;xl<sub>egg</sub>&rdquo; below).  New units can be added to the file [./Data/food_units.txt](./Data/food_units.txt).



### Example: My Shopping Trip



Here&rsquo;s an example of describing some different kinds of food, along with
data on food prices.  This is all just based on a trip I took to the
grocery store, except that I&rsquo;ve used the USDA database to look up FDC
numbers.  Note that we may need extra information to map some units
into weights.  For example, I still need to weigh a crumpet.



#### Trip to Monterey Market



In [1]:
import pandas as pd

df = pd.read_csv('./Data/ligons_shopping.csv',index_col=0)
df

### A Second Example: Villages in South India



Information on prices for different goods is found in a collection
  of `csv` files in [./Data](./Data).  You can generate additional files by
  using a spreadsheet and exporting to the appropriate file format,
  then putting that file in the [./Data>](./Data>)directory.  These files should
  have the same columns and format as the example above.

Here are some goods for which prices and quantities consumed were
recorded in a survey conducted by the International Crops Research
Institute of the Semi-Arid Tropics of several villages in South
India in the 1970s & early 1980s.  Local prices for these goods were
very low, but I&rsquo;ve found more recent on-line prices to record here.



In [1]:
import pandas as pd

df = pd.read_csv("./Data/icrisat_foods.csv",   # Prices for food consumed in Indian ICRISAT villages
                 dtype={'Price':float,
                        'Quantity':float,
                        'FDC':int})  
df

### Another Example: Stigler's Foods



In his 1945 paper George Stigler constructed a subsistence diet
chosen from 14 different goods (see Table B in [Stigler 1945](https://www.jstor.org/stable/pdf/1231810.pdf)), with
prices reported for the years 1939 & 1944.  

I&rsquo;ve looked up more recent prices for these same goods, and recorded
these at
[https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/](https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/).  

The code below allows us to collect data on different kinds of food
with their prices from google spreadsheets.

In this case, we use a function from a module I&rsquo;ve written,
 `eep153_tools.sheets`, to read the price data for the
Stigler goods.



In [1]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets("1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A")['Table B']

df

### Units & Prices



Now, the prices we observe can be for lots of different quantities and
 units.  The FDC database basically wants everything in either hundreds
 of grams (hectograms) or hundreds of milliliters (deciliters).  

Sometimes this conversion is simple; if the price we observe is for
something that weighs two kilograms, that&rsquo;s just 20 hectograms.
Different systems of weights and volumes are also easy; a five pound
bag of flour is approximately 22.68 hectograms.  

Othertimes things are more complicated.  If you observe the price of a
dozen donuts, that needs to be converted to hectograms, for example.  

A function `units` in the [fdc](fooddatacentral.py) module accomplishes this conversion
for many different units, using the `python` [pint module](https://pint.readthedocs.io/en/latest/).  A file
[./Data/food<sub>units.txt</sub>](Data/food_units.txt) can be edited to deal with odd cases such as
donuts, using a format described in the `pint` [documentation](https://pint.readthedocs.io/en/latest/defining.html). 

Here&rsquo;s an example of the usage of `fooddatacentral.units`:



In [1]:
# Try your own quantities and units.
# If units are missing try adding to ./Data/food_units.txt

print(fdc.units(5,'lbs'))
print(fdc.units(1,'gallon'))
print(fdc.units(2,'tea_bag'))
print(fdc.units(12,'donut'))

Now, use the `units` function to convert all foods to either
 deciliters or hectograms, to match FDC database:



In [1]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices

## Mapping to Nutrients



Next we want to build the matrix $A$, which maps quantities of food
 into nutrients.  We have a list of foods with prices.  Do lookups on USDA database
 to get nutritional information.



In [1]:
import fooddatacentral as fdc

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

D = pd.DataFrame(D,dtype=float)

D

## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements, and construct the vectors
$b_{min}$ and $b_{max}$.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf](https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf)

I&rsquo;ve put some of these data into a google spreadsheet at
[https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/](https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/). 
Note that we&rsquo;ve tweaked the nutrient labels to match those in the FDC
data.

We&rsquo;ve broken down the requirements into three different tables.  The
first is *minimum* quantities that we need to  satisfy.  For example,
this table tells us that a 20 year-old female needs at least 46 grams
of protein per day.



In [1]:
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
bmin

This next table specifies *maximum* quantities.  Our 20 year-old
female shouldn&rsquo;t have more than 2300 milligrams of sodium per day.



In [1]:
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
bmax

## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$



### Objective function ($p$)



If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.



In [1]:
p = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(p.index.tolist()).intersection(D.columns.tolist()))
p = p[use]

p

### Nutrient Mapping Matrix ($A$)



The matrix $A$ maps a bag of groceries $x$ into nutrients, but we
don&rsquo;t need to keep track of nutrients for which we don&rsquo;t have
contraints.



In [1]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

### Constraint vector ($b$)



Finally, the right hand side vector $b$ in the expression
$$
    Ax\geq b
$$



In [1]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b

## Solving the problem



First, we find a solution to the problem



In [1]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

## Choose sex/age group!
group = "F 19-30"

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(p, -A, -b[group], method='interior-point')

result

Let&rsquo;s interpret this.  Start with the cost of the solution:



In [1]:
print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

Next, what is it we&rsquo;re actually eating?



In [1]:
# Put back into nice series
diet = pd.Series(result.x,index=p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

Given this diet, what are nutritional outcomes?



In [1]:
tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
tab

Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [1]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())